In [1]:
import pandas as pd
import psycopg2 as pg
import os
import requests
import json
from io import StringIO

def importSymbols(dburl,logger=None):
    try:
        symbolsLoaded = False
        r = requests.get('https://api.iextrading.com/1.0/ref-data/symbols')
        symboljson = r.json()
        symboldf = pd.DataFrame(symboljson)
        symbolcsv = symboldf[symboldf.isEnabled==True][['name','symbol']].to_csv(header=False,index=False)
        conn = pg.connect(dburl)
        cur = conn.cursor()
        symbols = StringIO(symbolcsv)
        cur.execute('truncate symbols')
        cur.copy_from(symbols,'symbols',sep=',')
        symbols.close()
        conn.commit()
        conn.close()
        symbolsLoaded = True
    except Exception as ex:
        if logger:
            logger.warn(ex)
        else:
            print(ex)
    else:
        return symbolsLoaded
    
importSymbols(os.getenv('DATABASE_URL'))



True

In [1]:
import pandas as pd
import psycopg2 as pg
import os
import requests
import json
from io import StringIO

In [18]:
conn = pg.connect("user=moshea dbname=moshea")
cur = conn.cursor()
cur.execute("select portfolio from user_portfolios where email = 'oshea00@gmail.com'")
rows = cur.fetchone()
conn.close()

In [19]:
with StringIO(rows[0]) as csv:
    df = pd.read_csv(csv)

In [20]:
df.head()

,ceo,chart,companyName,description,exchange,industry,issueType,lastPrice,sector,ticker,website,weight
0,Timothy D. Cook,"[{'date': '2018-04-23', 'open': 166.194, 'high...",Apple Inc.,"Apple Inc is designs, manufactures and markets...",Nasdaq Global Select,Computer Hardware,cs,186.31,Technology,AAPL,http://www.apple.com,0.0%
1,Virginia M. Rometty,"[{'date': '2018-04-23', 'open': 143.4377, 'hig...",International Business Machines Corporation,International Business Machines Corp offers a ...,New York Stock Exchange,Application Software,cs,144.08,Technology,IBM,http://www.ibm.com,0.0%
2,NaN,"[{'date': '2018-04-23', 'open': 43.88, 'high':...",General Mills Inc.,General Mills Inc is a global manufacturer and...,New York Stock Exchange,Consumer Packaged Goods,cs,41.82,Consumer Defensive,GIS,http://www.generalmills.com,0.0%
